Extract Snotel location data from Snobal run results

In [ ]:
import datetime
import xarray as xr

from pathlib import PurePath, Path
from dask_cluster import CHPC, start_cluster

In [ ]:
"""
38.89433 -106.95300	330628	4306863 0  # Butte
39.01522 -107.04877	322624	4320462 0  # Schofield
38.99077 -106.75422	348074	4317216 0  # Upper Taylor

"""
SNOTEL = {
    'Butte': {
        'lon': 330628,
        'lat': 4306863,
    },
    'Schofield': {
        'lon': 322624,
        'lat': 4320462,
    },
    'Taylor': {
        'lon': 348074,
        'lat': 4317216,
    },
}

In [ ]:
client = start_cluster(4, 8)

In [ ]:
home = PurePath(f'{Path.home()}/scratch/iSnobal/runs/run201[7,8]*')
snow_nc = 'snow.nc'

## Snobal 

In [ ]:
snow = xr.open_mfdataset(
    str(home.joinpath(snow_nc)),
    parallel=True,
).sel(time=datetime.time(23))

In [ ]:
butte = snow.sel(x=SNOTEL['Butte']['lon'], y=SNOTEL['Butte']['lat'], method='nearest')
schofield = snow.sel(x=SNOTEL['Schofield']['lon'], y=SNOTEL['Schofield']['lat'], method='nearest')
taylor = snow.sel(x=SNOTEL['Taylor']['lon'], y=SNOTEL['Taylor']['lat'], method='nearest')

## Write compressed

In [ ]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in taylor.data_vars}

taylor.to_netcdf(
    PurePath(f'{Path.home()}/scratch/iSnobal/taylor.zarr').as_posix(), 
    encoding=encoding,
    compute=True
)